In [ ]:

import pandas as pd

# Load the two datasets
gdp_data = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_2.csv', skiprows=4)
migration_data = pd.read_csv('API_SM.POP.NETM_DS2_en_csv_v2_33.csv', skiprows=4)

# Clean and reshape the datasets

# Keep relevant columns (Country Code, Country Name, and yearly data)
gdp_data_clean = gdp_data.drop(columns=["Indicator Name", "Indicator Code", "Unnamed: 68"])
migration_data_clean = migration_data.drop(columns=["Indicator Name", "Indicator Code", "Unnamed: 68"])

# Melt datasets to have 'Year' as a column
gdp_melted = gdp_data_clean.melt(id_vars=["Country Name", "Country Code"], 
                                 var_name="Year", value_name="GDP")
migration_melted = migration_data_clean.melt(id_vars=["Country Name", "Country Code"], 
                                             var_name="Year", value_name="Net Migration")

# Merge datasets on Country Code, Country Name, and Year
merged_data = pd.merge(gdp_melted, migration_melted, 
                       on=["Country Name", "Country Code", "Year"], 
                       how="inner")

# Convert Year to numeric for proper analysis
merged_data["Year"] = pd.to_numeric(merged_data["Year"], errors="coerce")

# Drop rows with NaN values (in GDP or Net Migration)
merged_data = merged_data.dropna(subset=["GDP", "Net Migration"])

# Save the merged data to a file for further analysis
merged_data.to_csv("merged_gdp_migration_data.csv", index=False)

# Calculate correlation for each country
country_correlations = merged_data.groupby("Country Name").apply(
    lambda group: group["GDP"].corr(group["Net Migration"])
).reset_index()

# Rename columns for clarity
country_correlations.columns = ["Country Name", "Correlation (GDP vs. Net Migration)"]

# Save the correlation results to a file
country_correlations.to_csv("country_correlations.csv", index=False)

# Display final message
print("Processing complete! Results saved as 'merged_gdp_migration_data.csv' and 'country_correlations.csv'.")
